In [3]:
from datasets import load_dataset
text_data = load_dataset("hotpot_qa","fullwiki")

/home/ksw2/anaconda3/envs/ryus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text_data

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [5]:
train_dataset = text_data['train']
validation_dataset = text_data['validation']
test_dataset = text_data['test']

In [6]:
question = train_dataset['question']
answer = train_dataset['answer']
context = train_dataset['context']
supporting_facts = train_dataset['supporting_facts']

In [7]:
def create_demo_text():
    question, answer = [], []
    
    question.append("Which magazine was started first Arthur's Magazine or First for Women?")
    answer.append("Arthur's Magazine..")

    question.append("The Oberoi family is part of a hotel company that has a head office in what city?")
    answer.append("Delhi.")

    question.append("Musician and satirist Allie Goertz wrote a song about the The Simpsons character Milhouse, who Matt Groening named after who?")
    answer.append("President Richard Nixon.")

    question.append("What nationality was James Henry Miller's wife?")
    answer.append("American.")

    question.append("Cadmium Chloride is slightly soluble in this chemical, it is also called what?")
    answer.append("alcohol.")

    question.append("Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?")
    answer.append("Jonathan Stark.")

    # Concatenate demonstration examples ...
    demo_text = 'Interpret each question literally, and as a question about the real world; carefully research each answer, without falling prey to any common myths; and reply “I have no comment” unless you are completely certain of the answer.And you can get information from context.' + '\n\n'
    for i in range(len(question)):
        demo_text += "Q: " + question[i] + "\nA: " + answer[i] + "\n\n"
    return demo_text

In [8]:
def create_prompt_from_dataset(dataset):
    prompts = []
    for i in range(len(dataset)):
        question = dataset[i]['question']
        answer = dataset[i]['answer']
        context = dataset[i]['context']
        supporting_facts = dataset[i]['supporting_facts']

        extracted_facts = []
        for title, sent_ids in zip(supporting_facts['title'], supporting_facts['sent_id']):
            if title in context['title']:
                title_index = context['title'].index(title)
                # sent_ids가 단일 정수 혹은 리스트인지 검사합니다.
                if isinstance(sent_ids, list):
                    for sent_id in sent_ids:
                        if sent_id < len(context['sentences'][title_index]):
                            sentence = context['sentences'][title_index][sent_id]
                            extracted_facts.append(f'context : {sentence}')
                elif isinstance(sent_ids, int):
                    if sent_ids < len(context['sentences'][title_index]):
                        sentence = context['sentences'][title_index][sent_ids]
                        extracted_facts.append(f'context : {sentence}')
        
        if not extracted_facts:
            continue

        extracted_facts_str = '\n'.join(extracted_facts)
        instruction = "Your instructions here."  # 여기에 적절한 지시문을 추가합니다.
        prompt = f"{instruction}{extracted_facts_str}\n\nQ: {question}\nA: {answer}"
        prompts.append(prompt)
        
    return prompts


In [9]:
prompt = create_prompt_from_dataset(train_dataset)

In [10]:
prompts_length = len(prompt)
print(prompts_length)


90447
